<a href="https://colab.research.google.com/github/Stepka/Peptides-Analyse/blob/master/Blastim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/drive')

default_path = "/drive/My Drive/Colab Notebooks/Blastim/"

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
from collections import Counter

peptides_df = pd.read_excel(default_path + 'data_set_test_1.xlsx', index_col=0)

In [3]:
peptides_df.head()

,Label,Fragment,S2,S3,S4,S5,S6,S7,S8,S9,S10,S11,S12,S13,S14,S15,S16,S17
Peptide,,,,,,,,,,,,,,,,,,
P1,Light,y8,294.998383,448.058533,422.948151,631.757263,587.825317,321.863556,400.590027,343.929901,485.230499,368.712128,822.797241,417.526123,254.342270,254.642609,357.015076,218.479324
P1,Light,y7,1419.231689,1171.287231,1042.433350,998.465210,1079.857788,945.395386,983.617004,990.707825,881.312622,1140.147583,1126.719849,1261.232422,1243.158447,855.177307,1029.394897,902.800537
P1,Light,y5,1035.721436,915.735291,535.818237,797.905823,719.815430,1027.282593,701.084351,808.346069,635.738770,761.494507,698.001404,929.305786,554.095642,952.229919,512.895386,1250.173340
P1,Heavy,y8,1741.223877,1821.417969,1945.423340,1902.768799,1681.411987,1790.363525,1503.520020,1830.634033,2016.296021,2087.350830,2091.277344,2017.161255,1623.241211,1781.993652,2057.386719,1644.980835
P1,Heavy,y7,5349.316406,5757.182129,5494.454590,6000.090332,5274.203125,5138.413086,5296.849121,5082.851074,5097.641602,6026.864746,6089.832031,4401.206543,6459.663086,4476.049805,4950.586426,4978.967773


In [0]:
# Outlier detection 

def detect_outliers(df, features):
    """
    Takes a dataframe df of features and returns a list of the indices
    corresponding to the observations containing outliers according
    to the Tukey method.
    """
    outlier_indices = []
    
    # iterate over rows
    for index, row in df.iterrows():
      values = row[features].values
#     for col in features:
      # 1st quartile (25%)
      Q1 = np.percentile(values, 25)
      # 3rd quartile (75%)
      Q3 = np.percentile(values, 75)
      # Interquartile range (IQR)
      IQR = Q3 - Q1
        
      # outlier step
      outlier_step = 0.5 * IQR
      
      outlier_list = np.where((values < Q1 - outlier_step) | (values > Q3 + outlier_step))
      
      outlier_indices.append(outlier_list)
      
    return outlier_indices 

In [0]:
outliers = detect_outliers(peptides_df, ["S2", "S3", "S4", "S5", "S6", "S7", "S8", "S9", "S10", "S11", "S12", "S13", "S14", "S15", "S16", "S17"])

In [6]:
outliers_mask = np.ones((len(outliers), 16))

for i in range(len(outliers)):
  for j in outliers[i][0]:
    outliers_mask.itemset((i, int(j)), 0)

outliers_mask[0]

array([1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0.])

In [0]:
outliers_df = peptides_df.copy()

column_dict = {"S2": 0, "S3": 1, "S4": 2, "S5": 3, "S6": 4, "S7": 5, "S8": 6, "S9": 7, "S10": 8, "S11": 9, "S12": 10, "S13": 11, "S14": 12, "S15": 13, "S16": 14, "S17": 15}

i = 0
for index, row in outliers_df.iterrows():
  outliers_df.iloc[[i], 2:] = row[2:].values * outliers_mask[i]
  i += 1
  
outliers_marked_df = outliers_df.applymap(lambda x: 'DISTORTED' if x == 0 else x)

outliers_marked_df.head(10)

outliers_marked_df.to_excel("anomalies.xlsx")

In [0]:
# replace distorted values with means of samples

outliers_corrected_df = outliers_df.copy()
outliers_corrected_df = outliers_df.applymap(lambda x: None if x == 0 else x)

outliers_corrected_df['mean'] = outliers_corrected_df.mean(axis=1)

for column in column_dict.keys():
  outliers_corrected_df[column].fillna(outliers_corrected_df['mean'], inplace=True)

outliers_marked_df.to_excel("corrected.xlsx")